https://github.com/Dominant7/EEG_Classify

### EA Paper benchmarking details

1. Only Training phase data was used - has complete marker info
2. Causal band-pass filter (50-order linear phase Hamming
window FIR filter designed by Matlab function fir1, with 6dB cut-off frequencies at [8, 30] Hz)
3. Extracted EEG signals between [0.5, 3.5] seconds

Perf evaluation
1. Leave-one-out CV - Picked one subject as the new subject combined EEG trails from all remaining subjects as training set to build the classifier

Note: For subjects 4 and 9 in Dataset 2a, the classification performance actually became worse

In [1]:
import numpy as np
from moabb.datasets import BNCI2014_001
from moabb.paradigms import MotorImagery, LeftRightImagery
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mne.decoding import CSP
from moabb.evaluations import CrossSubjectEvaluation
from sklearn.pipeline import make_pipeline
from scipy import signal

In [2]:
from scipy import signal

def causal_bandpass_filter(data, lowcut=8, highcut=30, fs=250, order=50):
    nyq = 0.5 * fs
    # Normalize the cutoff frequencies (Matlab's fir1 expects normalized cutoff frequencies
    low = lowcut / nyq
    high = highcut / nyq
    # Design the FIR filter. Note: order+1 coefficients are returned to match Matlab's fir1 which returns n+1 taps.
    b = signal.firwin(order + 1, [low, high], window='hamming', pass_zero=False)
    # Apply the filter causally using lfilter (this introduces a constant delay).
    filtered_data = signal.lfilter(b, [1.0], data)
    return filtered_data

In [ ]:
dataset = BNCI2014_001()
paradigm = LeftRightImagery()
X = []
y = []
metadata = []
for i in range(1, 10):
    # print(i)
    xi, yi, metai = paradigm.get_data(dataset=dataset, subjects=[i])
    
    mask = metai['session'] != '1test'
    xi = xi[mask]
    yi = yi[mask]
    metai = metai[mask]
    xi = np.array([
        [causal_bandpass_filter(channel, lowcut=8, highcut=30, fs=250, order=50)
         for channel in trial]
        for trial in xi])
    X.append(xi)
    y.append(yi)
    metadata.append(metai)

/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~

In [20]:
print(X[0].shape)

(144, 22, 1001)


In [21]:
print(y[0])

['right_hand' 'left_hand' 'left_hand' 'right_hand' 'right_hand'
 'left_hand' 'left_hand' 'left_hand' 'right_hand' 'right_hand' 'left_hand'
 'left_hand' 'left_hand' 'right_hand' 'left_hand' 'right_hand'
 'right_hand' 'left_hand' 'right_hand' 'left_hand' 'right_hand'
 'right_hand' 'right_hand' 'left_hand' 'left_hand' 'left_hand'
 'right_hand' 'left_hand' 'left_hand' 'right_hand' 'left_hand' 'left_hand'
 'right_hand' 'left_hand' 'right_hand' 'right_hand' 'left_hand'
 'right_hand' 'left_hand' 'left_hand' 'right_hand' 'left_hand'
 'right_hand' 'right_hand' 'right_hand' 'right_hand' 'right_hand'
 'left_hand' 'right_hand' 'left_hand' 'left_hand' 'left_hand' 'right_hand'
 'right_hand' 'left_hand' 'right_hand' 'left_hand' 'right_hand'
 'left_hand' 'right_hand' 'right_hand' 'right_hand' 'left_hand'
 'left_hand' 'right_hand' 'left_hand' 'right_hand' 'right_hand'
 'left_hand' 'left_hand' 'right_hand' 'left_hand' 'right_hand' 'left_hand'
 'left_hand' 'right_hand' 'left_hand' 'left_hand' 'left_hand'

In [13]:
print(metadata[0]['session'].value_counts())

session
0train    144
Name: count, dtype: int64


In [14]:
print(X[0].shape[0])

144


In [15]:
X_aligned = []
def EA(X):
    covs = np.array([trial @ trial.T for trial in X])  # shape [n_trials, n_channels, n_channels]
    mean_cov = np.mean(covs, axis=0)
    # Eigen-decomposition of mean_cov
    eigvals, eigvecs = np.linalg.eigh(mean_cov)
    D_inv_sqrt = np.diag(eigvals**(-0.5))
    R_inv_sqrt = eigvecs @ D_inv_sqrt @ eigvecs.T

    # Align each trial
    xaligned = np.array([R_inv_sqrt @ trial for trial in X])
    X_aligned.append(xaligned)

for i in X:
    EA(i)


In [16]:
print(X_aligned[0].shape)

(144, 22, 1001)


In [17]:
n_components = 4
csp = CSP(n_components=n_components, reg=None, log=True, norm_trace=False)
lda = LinearDiscriminantAnalysis()

In [18]:
n_components = 4
accuracies = []

# Assuming:
# X_aligned = [subject1_data, ..., subject9_data] where each subject_data.shape = (288, 22, 1001)
# y = [subject1_labels, ..., subject9_labels] where each labels.shape = (288,)

for subj_idx in range(len(X_aligned)):
    # print(subj_idx)
    # Split data into train/test using leave-one-subject-out
    X_test = X_aligned[subj_idx]
    y_test = y[subj_idx]
    
    # Concatenate data from other subjects
    X_train = np.concatenate([X_aligned[i] for i in range(len(X_aligned)) if i != subj_idx], axis=0)
    y_train = np.concatenate([y[i] for i in range(len(y)) if i != subj_idx], axis=0)

    # Create pipeline
    pipeline = Pipeline([
        ('CSP', CSP(n_components=n_components, reg=None, log=True, norm_trace=False)),
        ('LDA', LinearDiscriminantAnalysis())
    ])

    # Fit and predict
    pipeline.fit(X_train, y_train)
    accuracy = pipeline.score(X_test, y_test)
    accuracies.append(accuracy)
    
for subj_idx in range(len(X_aligned)):
    print(f"Subject {subj_idx+1} Test Accuracy: {accuracies[subj_idx]:.2f}")

print(f"\nMean Cross-Validation Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")


Computing rank from data with rank=None
    Using tolerance 0.17 (2.2e-16 eps * 22 dim * 3.4e+13  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=left_hand covariance using EMPIRICAL
Done.
Estimating class=right_hand covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.17 (2.2e-16 eps * 22 dim * 3.4e+13  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=left_hand covariance using EMPIRICAL
Done.
Estimating class=right_hand covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.17 (2.2e-16 eps * 22 dim * 3.4e+13  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating cla